In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [83]:
original_df = pd.read_csv(basedir + filenames[0], encoding='euc-kr')

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
basedir = './토지특성정보/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
df = pd.read_csv(basedir + filenames[0], encoding='euc-kr', usecols=[2,3,5,7,9,11,12,14,18,20,22])

In [48]:
print(df.shape)
df.head()

(7468497, 11)


,법정동명,대장구분코드,지번,기준년도,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
0,서울특별시 종로구 청운동,1.0,1,2013,8,15622.1,13,0,2,4,6
1,서울특별시 종로구 청운동,1.0,1,2014,8,15622.1,13,0,2,4,6
2,서울특별시 종로구 청운동,1.0,1,2015,8,15622.1,13,0,2,4,6
3,서울특별시 종로구 청운동,1.0,1,2016,8,15622.1,13,0,2,4,6
4,서울특별시 종로구 청운동,1.0,1,2017,8,15622.1,13,0,2,4,6


In [49]:
def landnum_modifier(x):
    # x is a string
    
    splitted = x.split('-')
    if len(splitted) == 1:
        return x+'-0'
    elif len(splitted) == 2:
        return x
    else:
        raise ValueError('myerror')

In [50]:
df['지번주소'] = df['법정동명'] + ' ' + df['지번'].apply(landnum_modifier)

In [51]:
df.head()

,법정동명,대장구분코드,지번,기준년도,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,지번주소
0,서울특별시 종로구 청운동,1.0,1,2013,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
1,서울특별시 종로구 청운동,1.0,1,2014,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
2,서울특별시 종로구 청운동,1.0,1,2015,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
3,서울특별시 종로구 청운동,1.0,1,2016,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
4,서울특별시 종로구 청운동,1.0,1,2017,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0


In [52]:
df.drop(columns=['법정동명', '지번'], inplace=True)

In [54]:
for addr in df['지번주소'].unique():
    addr_df = df[df['지번주소'] == addr]
    if addr_df['대장구분코드'].nunique() > 1:
        display(addr_df)
        raise ValueError('myerror')

,대장구분코드,기준년도,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,지번주소
8,1.0,2013,14,21.4,13,0,2,7,0,서울특별시 종로구 청운동 1-1
9,1.0,2014,14,21.4,13,0,2,7,0,서울특별시 종로구 청운동 1-1
10,1.0,2015,14,21.4,13,0,2,7,0,서울특별시 종로구 청운동 1-1
11,1.0,2016,14,21.4,13,0,2,7,0,서울특별시 종로구 청운동 1-1
12,1.0,2017,14,21.4,13,0,2,7,0,서울특별시 종로구 청운동 1-1
13,1.0,2018,14,21.4,13,0,2,7,0,서울특별시 종로구 청운동 1-1
14,1.0,2019,14,21.4,13,0,2,7,0,서울특별시 종로구 청운동 1-1
15,1.0,2020,14,21.4,13,0,0,0,0,서울특별시 종로구 청운동 1-1
5736,2.0,2013,5,254940.0,44,0,5,7,4,서울특별시 종로구 청운동 1-1
5737,2.0,2014,5,254940.0,44,0,5,7,4,서울특별시 종로구 청운동 1-1


ValueError: myerror

In [55]:
df.drop_duplicates(subset=['지번주소', '기준년도', '대장구분코드'], keep='first', inplace=True)

In [ ]:
# NaN 어떻게 채울 것인지, 사이사이 비어있는 년도들은 어떻게 채울 것인지.

In [56]:
main_data_df = pd.read_csv('main_data_ver2.1.csv')

In [57]:
print(df.shape)
df.head()

(7455161, 10)


,대장구분코드,기준년도,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,지번주소
0,1.0,2013,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
1,1.0,2014,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
2,1.0,2015,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
3,1.0,2016,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
4,1.0,2017,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0


In [58]:
selected_df = df[df['지번주소'].isin(main_data_df['지번주소'].unique())].reset_index(drop=True)
print(selected_df.shape)
selected_df.head()

(868104, 10)


,대장구분코드,기준년도,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,지번주소
0,1.0,2013,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
1,1.0,2014,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
2,1.0,2015,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
3,1.0,2016,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
4,1.0,2017,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0


In [59]:
selected_df.rename(columns={'기준년도':'년'}, inplace=True)
selected_df.head()

,대장구분코드,년,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,지번주소
0,1.0,2013,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
1,1.0,2014,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
2,1.0,2015,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
3,1.0,2016,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0
4,1.0,2017,8,15622.1,13,0,2,4,6,서울특별시 종로구 청운동 1-0


In [63]:
selected_df = selected_df[['지번주소', '대장구분코드', '년', '지목코드', '토지면적', '용도지역코드1', '용도지역코드2',
                           '지형높이코드', '지형형상코드', '도로접면코드']]

In [64]:
selected_df.head()

,지번주소,대장구분코드,년,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
0,서울특별시 종로구 청운동 1-0,1.0,2013,8,15622.1,13,0,2,4,6
1,서울특별시 종로구 청운동 1-0,1.0,2014,8,15622.1,13,0,2,4,6
2,서울특별시 종로구 청운동 1-0,1.0,2015,8,15622.1,13,0,2,4,6
3,서울특별시 종로구 청운동 1-0,1.0,2016,8,15622.1,13,0,2,4,6
4,서울특별시 종로구 청운동 1-0,1.0,2017,8,15622.1,13,0,2,4,6


In [66]:
selected_df['대장구분코드'].value_counts()

1.0    863941
2.0      2688
3.0      1011
7.0       256
5.0       176
8.0        24
4.0         8
Name: 대장구분코드, dtype: int64

In [68]:
for code in selected_df['대장구분코드'].unique():
    print(code)
    lt_df = selected_df[selected_df['대장구분코드'] == code]
    display(lt_df.head())


1.0


,지번주소,대장구분코드,년,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
0,서울특별시 종로구 청운동 1-0,1.0,2013,8,15622.1,13,0,2,4,6
1,서울특별시 종로구 청운동 1-0,1.0,2014,8,15622.1,13,0,2,4,6
2,서울특별시 종로구 청운동 1-0,1.0,2015,8,15622.1,13,0,2,4,6
3,서울특별시 종로구 청운동 1-0,1.0,2016,8,15622.1,13,0,2,4,6
4,서울특별시 종로구 청운동 1-0,1.0,2017,8,15622.1,13,0,2,4,6


2.0


,지번주소,대장구분코드,년,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
368,서울특별시 종로구 청운동 4-35,2.0,2013,5,8043.0,44,0,5,7,8
369,서울특별시 종로구 청운동 4-35,2.0,2014,5,8043.0,44,0,5,7,8
370,서울특별시 종로구 청운동 4-35,2.0,2015,5,8043.0,44,0,5,7,8
371,서울특별시 종로구 청운동 4-35,2.0,2016,5,8043.0,44,43,5,7,8
372,서울특별시 종로구 청운동 4-35,2.0,2017,5,8043.0,44,43,5,7,8


3.0


...

,지번주소,대장구분코드,년,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
269826,서울특별시 노원구 상계동 6024-102,8.0,2013,8,290.6,14,0,2,3,8
269827,서울특별시 노원구 상계동 6024-102,8.0,2014,8,290.6,14,0,2,3,8
269828,서울특별시 노원구 상계동 6024-102,8.0,2015,8,290.6,13,0,2,3,8
269829,서울특별시 노원구 상계동 6024-102,8.0,2016,8,290.6,13,0,2,3,8
269830,서울특별시 노원구 상계동 6024-102,8.0,2017,8,290.6,13,0,2,3,8


4.0


,지번주소,대장구분코드,년,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
661670,서울특별시 관악구 봉천동 16-1,4.0,2013,8,100.9,15,0,2,4,10
661671,서울특별시 관악구 봉천동 16-1,4.0,2014,8,100.9,15,0,2,4,10
661672,서울특별시 관악구 봉천동 16-1,4.0,2015,8,100.9,15,0,2,4,10
661673,서울특별시 관악구 봉천동 16-1,4.0,2016,8,100.9,15,0,2,4,10
661674,서울특별시 관악구 봉천동 16-1,4.0,2017,8,100.9,15,0,2,4,10


5.0


,지번주소,대장구분코드,년,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
723653,서울특별시 서초구 신원동 690-0,5.0,2013,8,28407.0,15,0,2,7,5
723654,서울특별시 서초구 신원동 690-0,5.0,2014,8,28407.0,15,0,2,7,5
723655,서울특별시 서초구 신원동 690-0,5.0,2015,8,28407.0,15,0,2,7,5
723656,서울특별시 서초구 신원동 690-0,5.0,2016,8,28407.0,15,0,2,7,5
723657,서울특별시 서초구 신원동 690-0,5.0,2017,8,28407.0,15,0,2,7,5


In [65]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1


In [69]:
main_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4559866 entries, 0 to 4559865
Data columns (total 26 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   시군구                          object 
 1   번지                           object 
 2   본번                           int64  
 3   부번                           int64  
 4   건물명                          object 
 5   전월세매매구분                      object 
 6   전용면적(㎡)                      float64
 7   계약년월                         int64  
 8   계약일                          int64  
 9   금액(만원)                       int64  
 10  월세(만원)                       int64  
 11  층                            int64  
 12  건축년도                         int64  
 13  도로명                          object 
 14  가격/면적                        float64
 15  year                         int64  
 16  month                        int64  
 17  시                            object 
 18  구                            object 
 19  

In [70]:
san_df = main_data_df[main_data_df['번지'].str.contains('산')]
san_df.head()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [71]:
prac_df = main_data_df.dropna()

In [72]:
san_df = prac_df[prac_df['번지'].str.contains('산')]
san_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소
245877,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201307,5,25000,0,9,1994,신림로11길 151,296.559905,2013,7,서울특별시,관악구,10.126631,5.692249,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9
245878,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201308,19,25500,0,3,1994,신림로11길 151,302.491103,2013,8,서울특별시,관악구,10.146434,5.712052,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9
245879,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201308,27,26400,0,16,1994,신림로11길 151,313.167260,2013,8,서울특별시,관악구,10.181119,5.746737,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9
246856,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201405,24,28000,0,9,1994,신림로11길 151,332.147094,2014,5,서울특별시,관악구,10.239960,5.805578,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9
246857,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201409,5,27800,0,10,1994,신림로11길 151,329.774614,2014,9,서울특별시,관악구,10.232791,5.798409,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9


In [74]:
def dtype_identifier(x):
    # x is a string
    
    splitted = x.split('-')
    
    try:
        return type(int(splitted[0]))
    except:
        return type(splitted[0])

In [75]:
prac_df['dtype'] = prac_df['번지'].apply(dtype_identifier)
prac_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소,dtype
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2,<class 'int'>
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2,<class 'int'>
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1,<class 'int'>
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1,<class 'int'>
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1,<class 'int'>


In [76]:
prac_df['dtype'].value_counts()

<class 'int'>    4505255
<class 'str'>       6524
Name: dtype, dtype: int64

In [77]:
str_df = prac_df[prac_df['dtype'] == str]
str_df.shape

(6524, 27)

In [78]:
str_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소,dtype
245877,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201307,5,25000,0,9,1994,신림로11길 151,296.559905,2013,7,서울특별시,관악구,10.126631,5.692249,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>
245878,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201308,19,25500,0,3,1994,신림로11길 151,302.491103,2013,8,서울특별시,관악구,10.146434,5.712052,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>
245879,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201308,27,26400,0,16,1994,신림로11길 151,313.167260,2013,8,서울특별시,관악구,10.181119,5.746737,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>
246856,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201405,24,28000,0,9,1994,신림로11길 151,332.147094,2014,5,서울특별시,관악구,10.239960,5.805578,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>
246857,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201409,5,27800,0,10,1994,신림로11길 151,329.774614,2014,9,서울특별시,관악구,10.232791,5.798409,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>


In [79]:
no_san_df = str_df[str_df['번지'].str.contains('산') == False]
no_san_df.shape

(4448, 27)

In [80]:
no_san_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소,dtype
340231,서울특별시 구로구 항동,가-242,242,0,하버라인3단지,매매,74.8300,201904,30,48600,0,16,2019,항동로 60,649.472137,2019,4,서울특별시,구로구,10.791379,6.476160,서울특별시 구로구 항동,아파트,아파트_매매,2.0,서울특별시 구로구 항동 가-242,<class 'str'>
340232,서울특별시 구로구 항동,가-242,242,0,하버라인3단지,매매,84.8800,201907,12,49000,0,6,2019,항동로 43,577.285580,2019,7,서울특별시,구로구,10.799576,6.358337,서울특별시 구로구 항동,아파트,아파트_매매,2.0,서울특별시 구로구 항동 가-242,<class 'str'>
340233,서울특별시 구로구 항동,가-238,238,0,하버라인4단지,매매,59.8600,201908,31,40000,0,12,2019,항동로 72,668.225860,2019,8,서울특별시,구로구,10.596635,6.504626,서울특별시 구로구 항동,아파트,아파트_매매,2.0,서울특별시 구로구 항동 가-238,<class 'str'>
340234,서울특별시 구로구 항동,가-241,241,0,한양수자인에듀힐즈,매매,84.9544,201910,29,49325,0,7,2019,항동로 42,580.605595,2019,10,서울특별시,구로구,10.806186,6.364072,서울특별시 구로구 항동,아파트,아파트_매매,2.0,서울특별시 구로구 항동 가-241,<class 'str'>
340235,서울특별시 구로구 항동,가-241,241,0,한양수자인에듀힐즈,매매,84.9591,201911,10,47400,0,2,2019,항동로 42,557.915515,2019,11,서울특별시,구로구,10.766378,6.324208,서울특별시 구로구 항동,아파트,아파트_매매,2.0,서울특별시 구로구 항동 가-241,<class 'str'>


In [81]:
no_ga_df = no_san_df[no_san_df['번지'].str.contains('가-') == False]
no_ga_df.shape

(3949, 27)

In [82]:
no_ga_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소,dtype
715393,서울특별시 서초구 신원동,BL-3,3,0,서초포레스타3단지,매매,114.81,201406,25,73226,0,2,2014,헌릉로8길 22,637.801585,2014,6,서울특별시,서초구,11.201306,6.458027,서울특별시 서초구 신원동,아파트,아파트_매매,2.0,서울특별시 서초구 신원동 BL-3,<class 'str'>
715394,서울특별시 서초구 신원동,BL-3,3,0,서초포레스타3단지,매매,114.81,201407,7,73226,0,2,2014,헌릉로8길 22,637.801585,2014,7,서울특별시,서초구,11.201306,6.458027,서울특별시 서초구 신원동,아파트,아파트_매매,2.0,서울특별시 서초구 신원동 BL-3,<class 'str'>
715395,서울특별시 서초구 신원동,BL-3,3,0,서초포레스타3단지,매매,114.81,201407,16,73226,0,1,2014,헌릉로8길 22,637.801585,2014,7,서울특별시,서초구,11.201306,6.458027,서울특별시 서초구 신원동,아파트,아파트_매매,2.0,서울특별시 서초구 신원동 BL-3,<class 'str'>
715396,서울특별시 서초구 신원동,BL-3,3,0,서초포레스타3단지,매매,114.39,201407,16,73050,0,1,2014,헌릉로8길 22,638.604773,2014,7,서울특별시,서초구,11.198899,6.459286,서울특별시 서초구 신원동,아파트,아파트_매매,2.0,서울특별시 서초구 신원동 BL-3,<class 'str'>
715397,서울특별시 서초구 신원동,BL-3,3,0,서초포레스타3단지,매매,114.39,201407,21,73050,0,1,2014,헌릉로8길 22,638.604773,2014,7,서울특별시,서초구,11.198899,6.459286,서울특별시 서초구 신원동,아파트,아파트_매매,2.0,서울특별시 서초구 신원동 BL-3,<class 'str'>


In [85]:
original_df['대장구분명'].value_counts()

일반             7309772
산               126189
가지번              13550
블럭지번(지구)          9996
블럭지번              4824
기타지번              2830
블럭지번(지구_롯트)       1030
가지번(부번세분)          304
Name: 대장구분명, dtype: int64

In [88]:
prac_df['onlystr'] = prac_df['번지'].apply(show_only_str)
prac_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소,dtype,onlystr
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2,<class 'int'>,
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2,<class 'int'>,
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1,<class 'int'>,
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1,<class 'int'>,
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1,<class 'int'>,


In [89]:
no_empty_df = prac_df[prac_df['onlystr'] != '']
no_empty_df.shape

(6524, 28)

In [90]:
no_empty_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소,dtype,onlystr
245877,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201307,5,25000,0,9,1994,신림로11길 151,296.559905,2013,7,서울특별시,관악구,10.126631,5.692249,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>,산
245878,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201308,19,25500,0,3,1994,신림로11길 151,302.491103,2013,8,서울특별시,관악구,10.146434,5.712052,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>,산
245879,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201308,27,26400,0,16,1994,신림로11길 151,313.167260,2013,8,서울특별시,관악구,10.181119,5.746737,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>,산
246856,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201405,24,28000,0,9,1994,신림로11길 151,332.147094,2014,5,서울특별시,관악구,10.239960,5.805578,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>,산
246857,서울특별시 관악구 신림동,산28-9,28,9,신동아,매매,84.3,201409,5,27800,0,10,1994,신림로11길 151,329.774614,2014,9,서울특별시,관악구,10.232791,5.798409,서울특별시 관악구 신림동,아파트,아파트_매매,2.0,서울특별시 관악구 신림동 산28-9,<class 'str'>,산


In [91]:
no_empty_df['onlystr'].value_counts()

BL       3603
산        2076
가         499
지구BL      346
Name: onlystr, dtype: int64

In [92]:
original_df['대장구분명'].value_counts()

일반             7309772
산               126189
가지번              13550
블럭지번(지구)          9996
블럭지번              4824
기타지번              2830
블럭지번(지구_롯트)       1030
가지번(부번세분)          304
Name: 대장구분명, dtype: int64

In [93]:
original_df['대장구분코드'].value_counts()

1.0    7309772
2.0     126189
3.0      13550
7.0       9996
5.0       4824
9.0       2830
8.0       1030
4.0        304
Name: 대장구분코드, dtype: int64

In [96]:
selected_df['대장구분코드'].value_counts()

1.0    863941
2.0      2688
3.0      1011
7.0       256
5.0       176
8.0        24
4.0         8
Name: 대장구분코드, dtype: int64

In [97]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,split_length,지번주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 655-2
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,2.0,서울특별시 강남구 개포동 658-1


In [100]:
nonan_df = main_data_df.dropna()

In [133]:
def landtype_allocator(x):
    # x is a string
    if pd.isna(x) == True:
        return '일반'
    
    x = x.replace('-', '')
    
    for i in range(10):
        x = x.replace('{}'.format(i), '')
    
    if x == '':
        return '일반'
    
    return x

In [134]:
main_data_df['대장구분명'] = main_data_df['번지'].apply(landtype_allocator)
main_data_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,지번주소,대장구분코드,대장구분명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1.0,일반
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1.0,일반
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1.0,일반
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1.0,일반
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1.0,일반


In [135]:
main_data_df['대장구분명'].value_counts()

일반       4553162
BL          3759
산           2091
가            508
지구BL         346
Name: 대장구분명, dtype: int64

In [146]:
def landtypecode_allocator(x):
    # x is a string
    
    if x == '일반':
        return 1
    elif x == 'BL':
        return 5
    elif x == '산':
        return 2
    elif x == '가':
        return 3
    elif '지구' in x:
        return 7
    else:
        print(x)
        print(x == '지구BL')
        raise ValueError('myerror')

In [147]:
main_data_df['대장구분코드'] = main_data_df['대장구분명'].apply(landtypecode_allocator)
main_data_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,지번주소,대장구분코드,대장구분명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반


In [148]:
main_data_df['대장구분코드'].value_counts()

1    4553162
5       3759
2       2091
3        508
7        346
Name: 대장구분코드, dtype: int64

In [39]:
del df, main_data_df

In [ ]:
def create_complete_land_plans_df_iteratively(df):
    import numpy as np
    import pandas as pd
    import gc
    
    df = df.copy()
    
    dfs_list = []
    count = 0
    for addr in df['지번주소'].unique():
        if (count%10000) == 0:
            gc.collect()
        
        addr_df = df[df['지번주소'] == addr]
        landtype = addr_df['대장구분명'].iloc[0]
        years = [i for i in range(2006, 2021)]
        [years.remove(y) for y in addr_df['년'].unique()]
        min_year = addr_df['년'].min()

        temp_dict = {}
        for col in df.columns:
            if col == '지번주소':
                temp_dict[col] = addr
            elif col == '대장구분명':
                temp_dict[col] = landtype
            elif col == '년':
                temp_dict[col] = years
            else:
                temp_dict[col] = np.nan

        add_df = pd.DataFrame(temp_dict)

        mini_concat_df = pd.concat([addr_df, add_df]).sort_values(['년']).fillna(method='ffill')
        
        min_year_row = mini_concat_df.loc[[mini_concat_df.index[min_year-2006]],:]
        
        min_year_concat_df = pd.concat([min_year_row for i in range(min_year-2006)])
        
        mini_concat_df.iloc[[i for i in range(min_year-2006)], 3:] = min_year_concat_df.iloc[:, 3:]
        
        mini_concat_df = mini_concat_df.fillna('해당없음')
        
        dfs_list.append(mini_concat_df)
        
        count += 1

    concat_df = pd.concat(dfs_list)
    
    return concat_df

In [ ]:
def parallelize(df, func, num_processors=12):
    #data_split = np.array_split(data, num_processors)
    
    data_split = []
    unique_addrs = df['지번주소'].unique()
    divided = int(len(unique_addrs) / num_processors)
    for i in range(num_processors):
        if i < (num_processors - 1):
            picked_addrs = unique_addrs[divided*i:divided*(i+1)]
        else:
            picked_addrs = unique_addrs[divided*i:]
        data_split.append(df[df['지번주소'].isin(picked_addrs)])
            
    pool = mp.Pool(num_processors)
    concat_df = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return concat_df

In [ ]:
%%time
completed_df = parallelize(selected_land_plans_df, create_complete_land_plans_df_iteratively)
print(completed_df.shape)
completed_df.head()

In [ ]:
completed_df.to_csv('land_plans_ver1.csv', index=False)

In [ ]:
completed_df.head(50)

In [ ]:
def create_complete_land_plans_df_reshape(df):
    df = df.copy()
    gc.collect()
    
    print('1')
    df = df.set_index(['지번주소', '대장구분명', '년']).stack(dropna=False)
    print('2')
    df = df.reset_index()
    print('3')
    df['binary'] = 1
    print('4')
    df = df.set_index(['지번주소', '대장구분명', 'level_3', 0, '년']).unstack()
    print('5')
    df[('binary', '2006')] = np.nan
    print('6')
    df = df.stack(dropna=False)
    print('7')
    df = df.reset_index()
    print('8')
    df = df[['지번주소', '대장구분명', 'level_3', 0, '년']]
    print('9')
    df = df.set_index(['지번주소', '대장구분명', '년', 'level_3']).unstack()
    print('10')
    
    return df

In [ ]:
# decided not to use it due to its super high memory consumption
#prac_df = create_complete_land_plans_df_reshape(selected_land_plans_df)